# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [50]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

gmaps.configure(api_key=g_key)
gmaps.figure()

from pprint import pprint

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [51]:
# Create dataframe
weather_df = pd.read_csv('../output_data/weather_df.csv')
weather_df = weather_df.drop(columns=["Unnamed: 0"])
weather_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,cockburn town,21.4612,-71.1419,82.40,61,40,17.27,TC,1619716336
1,quepos,9.4236,-84.1652,75.20,73,75,5.75,CR,1619716402
2,nanortalik,60.1432,-45.2371,38.70,78,16,5.77,GL,1619716294
3,codrington,-38.2667,141.9667,53.24,91,22,5.50,AU,1619716402
4,eisiskes,54.1667,25.0000,55.99,47,0,5.75,LT,1619716403
...,...,...,...,...,...,...,...,...,...
554,barreirinhas,-2.7469,-42.8264,83.25,76,96,6.51,BR,1619716751
555,lawton,34.6087,-98.3903,66.20,68,1,17.27,US,1619716751
556,san policarpo,12.1791,125.5072,76.82,81,90,1.32,PH,1619716711
557,raga,8.4596,25.6780,78.13,64,96,4.52,SS,1619716752


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [52]:
location = weather_df[["Lat","Lng"]]
humidity = weather_df["Humidity"].astype(int)

# Plot Heatmap
fig = gmaps.figure()

#create heat layer
heat_layer = gmaps.heatmap_layer(location,
                                 weights=humidity,
                                 dissipating=False,
                                 max_intensity=100,
                                 point_radius=1)

# Add layer
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [53]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions.

new_df = pd.DataFrame(weather_df, columns = ["City","Max Temp","Wind Speed","Cloudiness"])

temp = (new_df["Max Temp"] <= 80) & (new_df["Max Temp"] > 70)
wind = (new_df["Wind Speed"] < 10)
cloud = (new_df["Cloudiness"] == 0)

new_weather_df = new_df[temp & wind & cloud]
new_weather_df

,City,Max Temp,Wind Speed,Cloudiness
10,chuy,70.81,4.52,0
23,cidreira,70.43,7.31,0
48,saint george,72.00,1.50,0
160,rafaela,75.00,4.00,0
192,mar del plata,75.20,9.22,0
211,lompoc,73.00,2.59,0
262,san vicente,76.98,1.83,0
273,aksu,73.54,5.48,0
299,san quintin,70.18,7.47,0
308,resistencia,77.00,5.75,0


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [64]:
merge_df = pd.merge(new_weather_df,weather_df, how = "left", on="City")

hotel_df = pd.DataFrame(merge_df, columns = ["City","Max Temp_x","Wind Speed_x","Cloudiness_x","Lat","Lng","Country"])
hotel_df["Hotel Name"] = ""

hotel_df

,City,Max Temp_x,Wind Speed_x,Cloudiness_x,Lat,Lng,Country,Hotel Name
0,chuy,70.81,4.52,0,-33.6971,-53.4616,UY,
1,cidreira,70.43,7.31,0,-30.1811,-50.2056,BR,
2,saint george,72.00,1.50,0,37.1041,-113.5841,US,
3,rafaela,75.00,4.00,0,-31.2503,-61.4867,AR,
4,mar del plata,75.20,9.22,0,-38.0023,-57.5575,AR,
5,lompoc,73.00,2.59,0,34.6391,-120.4579,US,
6,san vicente,76.98,1.83,0,-26.6167,-54.1333,AR,
7,aksu,73.54,5.48,0,41.1231,80.2644,CN,
8,san quintin,70.18,7.47,0,30.4833,-115.9500,MX,
9,resistencia,77.00,5.75,0,-27.4606,-58.9839,AR,


In [69]:
# Set hotel name list
hotel_name = []

for city in range(len(hotel_df)):
    lat = hotel_df.loc[city]['Lat']
    lng = hotel_df.loc[city]['Lng']

    #Set parameters
    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "keywords" : "hotel locations",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    response = requested.json()
    try:
        hotel_name.append(response['results'][0]['name'])
    except:
        hotel_name.append("")
        
hotel_df["Hotel Name"] = hotel_name
hotel_df = hotel_df.dropna(how='any')
hotel_df

,City,Max Temp_x,Wind Speed_x,Cloudiness_x,Lat,Lng,Country,Hotel Name
0,chuy,70.81,4.52,0,-33.6971,-53.4616,UY,Chuy
1,cidreira,70.43,7.31,0,-30.1811,-50.2056,BR,CIDREIRA
2,saint george,72.00,1.50,0,37.1041,-113.5841,US,St. George
3,rafaela,75.00,4.00,0,-31.2503,-61.4867,AR,Rafaela
4,mar del plata,75.20,9.22,0,-38.0023,-57.5575,AR,Mar del Plata
5,lompoc,73.00,2.59,0,34.6391,-120.4579,US,Lompoc
6,san vicente,76.98,1.83,0,-26.6167,-54.1333,AR,San Pedro
7,aksu,73.54,5.48,0,41.1231,80.2644,CN,Aksu Prefecture
8,san quintin,70.18,7.47,0,30.4833,-115.9500,MX,Lázaro Cárdenas
9,resistencia,77.00,5.75,0,-27.4606,-58.9839,AR,Resistencia


In [89]:
# NOTE: Do not change any of the code in this cell

# Assign the marker layer to a variable
locations = hotel_df[["Lat", "Lng"]]


# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

markers = gmaps.marker_layer(locations,info_box_content = hotel_info, display_info_box=True)

In [90]:
#hotel_info_layer = gmaps.symbol_layer(locations, info_box_content = hotel_info, display_info_box=True)
# Add marker layer ontop of heat map

fig = gmaps.figure()

fig.add_layer(heat_layer)
fig.add_layer(markers)

fig

Figure(layout=FigureLayout(height='420px'))